In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from nn.cnn.lenet5 import mnist_utils

In [2]:
image_size = 28  # 图片尺寸
num_labels = 10  #每张图片的输出标签个数
num_channels = 1  #图片通道数（类似RGB）

# leNet5
batch_size = 20  #批次大小
kernelSize = 5  #卷积模板尺寸
depth1Size = 6  #第一卷积层深度
depth2Size = 16  #第二卷积层深度
padding = "SAME"  #填充方式
convStride = 1  #卷积层移动单位
poolStride = 2  #池化层移动单位
poolFilterSize = 2  #池化层过滤器大小
FC1HiddenUnit = 360  #全连接第一层隐藏单元
FC2HiddenUnit = 256  #全连接第二层隐藏单元
learningRate = 0.0001  #初始学习率
LEARNING_RATE_DECAY = 0.99  #衰减率

In [3]:
# 准备数据
X_train, Y_train = mnist_utils.get_train_data()

train_data, test_data, train_labels, test_labels = train_test_split(X_train, Y_train, test_size=0.2, random_state=0)

test_data, valid_data, test_labels, valid_labels = train_test_split(test_data, test_labels, test_size=0.5,
                                                                    random_state=0)
del Y_train, X_train

print('\n-------------------数据集信息-----------------------------\n')
print('Training set   :', train_data.shape, train_labels.shape)
print('Validation set :', valid_data.shape, valid_labels.shape)
print('Test set       :', test_data.shape, test_labels.shape)
print('\n---------------------------------------------------------\n')



-------------------数据集信息-----------------------------

Training set   : (48000, 28, 28, 1) (48000, 10)
Validation set : (6000, 28, 28, 1) (6000, 10)
Test set       : (6000, 28, 28, 1) (6000, 10)

---------------------------------------------------------



定义辅助函数

In [4]:
def output_size_pool(input_size, conv_filter_size, pool_filter_size, padding, conv_stride, pool_stride):
    """计算卷积池化之后图片的尺寸"""
    if padding == 'SAME':
        padding = 1.00
    elif padding == 'VALID':
        padding = 0.00
    else:
        return None
    # 卷积后尺寸
    output_1 = (((input_size - conv_filter_size + 2 * padding) / conv_stride) + 1.00)
    # 池化后尺寸
    output_2 = (((output_1 - pool_filter_size + 2 * padding) / pool_stride) + 1.00)
    # 卷积后尺寸
    output_3 = (((output_2 - conv_filter_size + 2 * padding) / conv_stride) + 1.00)
    # 池化后尺寸
    output_4 = (((output_3 - pool_filter_size + 2 * padding) / pool_stride) + 1.00)
    return int(output_4)


def accuracy(predictions, labels):
    """计算正确率（百分比"""
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

def weightBuilder(shape, name):
    """定义变量"""
    return tf.Variable(tf.truncated_normal(shape, stddev=0.01), name=name)


def biasesBuilder(shape, name):
    """定义bias项"""
    return tf.Variable(tf.constant(1.0, shape=shape), name=name)


def conv2d(x, W, name):
    """定义二维卷积"""
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding=padding, name=name)


def maxPool_2x2(x, name):
    """定义池化"""
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding=padding, name=name)

定义lenet5网络

In [5]:
# 卷积池化之后的图像尺寸
finalImageSize = output_size_pool(input_size=image_size, conv_filter_size=kernelSize,
                                  pool_filter_size=poolFilterSize, padding=padding,
                                  conv_stride=convStride, pool_stride=poolStride)
graph = tf.Graph()
with graph.as_default():
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    # 验证数据集 定义为常量
    tf_valid_dataset = tf.constant(valid_data)
    # 测试数据集 定义为常量
    tf_test_dataset = tf.constant(test_data)

    with tf.name_scope('convolution1') as scope:
        # weight & biases
        C1_w = weightBuilder([kernelSize, kernelSize, num_channels, depth1Size], "C1_w")
        C1_b = biasesBuilder([depth1Size], "C1_b")

    with tf.name_scope('convolution2') as scope:
        C2_w = weightBuilder([kernelSize, kernelSize, depth1Size, depth2Size], "C2_w")
        C2_b = biasesBuilder([depth2Size], "C2_b")

    with tf.name_scope('fullyConct1') as scope:
        FC1_w = weightBuilder([finalImageSize * finalImageSize * depth2Size, FC1HiddenUnit], "FC1_w")
        FC1_b = biasesBuilder([FC1HiddenUnit], "FC1_b")
        keep_prob = tf.placeholder(dtype=tf.float32, name="keepProb")

    with tf.name_scope('fullyConct2') as scope:
        FC2_w = weightBuilder([FC1HiddenUnit, FC2HiddenUnit], "FC2_w")
        FC2_b = biasesBuilder([FC2HiddenUnit], "FC2_b")

    with tf.name_scope('fullyConct3') as scope:
        FC3_w = weightBuilder([FC2HiddenUnit, num_labels], "FC3_w")
        FC3_b = biasesBuilder([num_labels], "FC3_b")

        #定义lenet5 前向传播
    def leNet5(data):
        # C1
        h_conv = tf.nn.relu(conv2d(data, C1_w, "conv1") + C1_b)
        # P2
        h_pool = maxPool_2x2(h_conv, "pool1")

        # C3
        h_conv = tf.nn.relu(conv2d(h_pool, C2_w, "conv2") + C2_b)
        # P4
        h_pool = maxPool_2x2(h_conv, "pool2")

        # reshape last conv layer
        shape = h_pool.get_shape().as_list()
        h_pool_reshaped = tf.reshape(h_pool, [shape[0], shape[1] * shape[2] * shape[3]])

        # F5
        h_FC1 = tf.nn.relu(tf.matmul(h_pool_reshaped, FC1_w) + FC1_b)
        h_FC1 = tf.nn.dropout(h_FC1, keep_prob=keep_prob)

        # F6
        h_FC2 = tf.nn.relu(tf.matmul(h_FC1, FC2_w) + FC2_b)

        # OUTPUT
        model = tf.matmul(h_FC2, FC3_w) + FC3_b
        return model


    # train computation
    logits = leNet5(tf_train_dataset)
    # 损失函数
    with tf.name_scope('loss') as scope:
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=tf_train_labels))

    # 指数衰减学习率
    global_step = tf.Variable(0, trainable=False)
    learning_rate = tf.train.exponential_decay(learningRate, global_step, train_data.shape[0] / batch_size,
                                               LEARNING_RATE_DECAY, staircase=True)
    # 优化
    optimizer = tf.train.AdamOptimizer(learning_rate=learningRate).minimize(loss)

    # 训练，验证，测试集的预测结果
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(leNet5(tf_valid_dataset))
    test_prediction = tf.nn.softmax(leNet5(tf_test_dataset))

训练并输出结果

In [6]:
# 循环轮数
num_epochs = 2
# 每一轮批次数
total_batch = int(train_data.shape[0] / batch_size)

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()

    for epoch in range(num_epochs):
        avg_cost = 0.
        print('{:6}|{:15}|{:15}|{:15}'.format("Step", "loss", "MiniBatch acc", "valid acc"))
        for step in range(total_batch):
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

            batch_data = train_data[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]

            feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels, keep_prob: 0.5}
            # train
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)

            # 计算平均损失值
            avg_cost += l / total_batch
            # 200批次进行输出
            if (step % 200 == 0):
                valid_pred = session.run(valid_prediction, feed_dict={keep_prob: 1})
                print('{:6}|{:15}|{:15}|{:15}'.format("{:d}".format(step),
                                                      "{:.9f}".format(l),
                                                      "{:.9f}".format(accuracy(predictions, batch_labels)),
                                                      "{:.9f}".format(accuracy(valid_pred, valid_labels))))
        # end for batch

        print("\nEpoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(avg_cost), "\n")

    test_pred = session.run(test_prediction, feed_dict={keep_prob: 1})
    print("Test 数据集正确率: %.6f%%" % accuracy(test_pred, test_labels))


Step  |loss           |MiniBatch acc  |valid acc      


0     |2.335149765    |10.000000000   |10.083333333   


200   |2.308820248    |5.000000000    |10.583333333   


400   |2.318854332    |10.000000000   |9.883333333    


600   |2.220359087    |15.000000000   |20.600000000   


...

1800  |0.148851067    |90.000000000   |93.216666667   


2000  |0.128434226    |95.000000000   |93.433333333   


2200  |0.293642402    |85.000000000   |93.750000000   



Epoch: 0002 cost= 0.315141845 



Test 数据集正确率: 94.316667%
